In [1]:
from fastai.vision import *

In [2]:
path = Path('data/ILSVRC/Data/CLS-LOC')

In [3]:
os.listdir(path)

['models', 'test', 'train', 'valid']

In [4]:
len(os.listdir(path/'train'))

1000

In [5]:
os.listdir(path/'train')[:5]

['n01440764', 'n01443537', 'n01484850', 'n01491361', 'n01494475']

In [6]:
len(os.listdir(path/'valid'))

1000

In [7]:
os.listdir(path/'valid')[:5]

['n01440764', 'n01443537', 'n01484850', 'n01491361', 'n01494475']

In [8]:
len(os.listdir(path/'test'))

100000

In [9]:
os.listdir(path/'test')[:5]

['ILSVRC2012_test_00000001.JPEG',
 'ILSVRC2012_test_00000002.JPEG',
 'ILSVRC2012_test_00000003.JPEG',
 'ILSVRC2012_test_00000004.JPEG',
 'ILSVRC2012_test_00000005.JPEG']

### for creating the file structure (only valid because train was correct)

In [10]:
#os.rename(path/'val', path/'valid')
#
#train_classes = os.listdir(path/'train')
#
#for line in open('data/valid.txt'):
#    valid_path = path/'valid'/line.split(' ')[0]
#    valid_class = line.split(' ')[1].replace('\n','')
#    real_class = train_classes[int(valid_class)]
#    if not os.path.isdir(path/'valid'/real_class):
#        os.mkdir(path/'valid'/real_class)
#    os.rename(valid_path, path/'valid'/real_class/line.split(' ')[0])

### Callback to save best epoch automatically

In [11]:
@dataclass
class TrackerCallback(LearnerCallback):
    "A `LearnerCallback` that keeps track of the best value in `monitor`."
    monitor:str='val_loss'
    mode:str='auto'
    
    def __post_init__(self):
        if self.mode not in ['auto', 'min', 'max']:
            warn(f'{self.__name__} mode {self.mode} is invalid, falling back to "auto" mode.')
            self.mode = 'auto'
        mode_dict = {'min': np.less, 'max':np.greater}
        mode_dict['auto'] = np.less if 'loss' in self.monitor else np.greater
        self.operator = mode_dict[self.mode]
    
    def on_train_begin(self, **kwargs:Any)->None:
        self.best = float('inf') if self.operator == np.less else -float('inf')
    
    def get_monitor_value(self):
        values = {'trn_loss':self.learn.recorder.losses[-1:][0].cpu().numpy(),
                  'val_loss':self.learn.recorder.val_losses[-1:][0]}
        for i, name in enumerate(self.learn.recorder.names[3:]):
            values[name]=learn.recorder.metrics[-1:][0][i] 
        if values.get(self.monitor) is None:
            warn(f'{self.__name__} conditioned on metric `{self.monitor}` which is not available. Available metrics are: {", ".join(map(str, self.learn.recorder.names[1:]))}')   
        return values.get(self.monitor)

In [12]:
@dataclass
class SaveModel(TrackerCallback):
    "A `LearnerCallback` that saves the model when monitored quantity is best."
    every:str='improvement'
    name:str='bestmodel'
    def __post_init__(self):
        if self.every not in ['improvement', 'epoch']:
            warn(f'SaveModel every {every} is invalid, falling back to "improvement".')
            self.every = 'improvement'
        super().__post_init__()
   
    def on_epoch_end(self, epoch, **kwargs:Any)->None:
        if self.every=="epoch": learn.save(f'{self.name}_{epoch}')
        else: #every="improvement"
            current = self.get_monitor_value()
            if current is not None and self.operator(current, self.best):
                self.best = current
                learn.save(f'{self.name}')
    
    def on_train_end(self, **kwargs):
        if self.every=="improvement": learn.load(f'{self.name}')

### create databunch from folder structure

In [13]:
data_bunch = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), size=299, num_workers=8, bs=64) \
                           .normalize(imagenet_stats)

In [16]:
data_bunch.show_batch(rows=2, figsize=(6,6))

In [ ]:
data_bunch.show_batch(rows=2, figsize=(6,6), ds_type=DatasetType.Valid)

### create learner with resnet50 (the pytorch model is already pretrained on imagenet)

In [12]:
learn = cnn_learner(data_bunch, models.resnet50, pretrained=True)

In [13]:
def metric(input, targs, k=5):
    return top_k_accuracy(input, targs, k)

In [14]:
learn.metrics=[metric]

In [ ]:
learn.freeze()

In [ ]:
apply_init(learn.model[1], nn.init.kaiming_normal_)

In [ ]:
learn.callback_fns.append(partial(SaveModel, every='epoch', monitor='val_loss'))

In [ ]:
learn.lr_find()

In [ ]:
lr = 2e-5

lrs = slice(lr/2, lr)

In [ ]:
learn.lr_range(lrs)

In [25]:
learn.fit_one_cycle(6, lrs, div_factor=25)

epoch,train_loss,valid_loss,metric,time


KeyboardInterrupt: 

In [12]:
cl, label, prob = learn.predict(img)

In [8]:
learn.export('export/export.pkl')

In [9]:
learn = load_learner('export')

In [12]:
# Load Imagenet Synsets
with open('data/imagenet_synsets.txt', 'r') as f:
    synsets = f.readlines()

# len(synsets)==1001
# sysnets[0] == background
synsets = [x.strip() for x in synsets]
splits = [line.split(' ') for line in synsets]
key_to_classname = {spl[0]:' '.join(spl[1:]) for spl in splits}

with open('data/imagenet_classes.txt', 'r') as f:
    class_id_to_key = f.readlines()

class_id_to_key = np.array([x.strip() for x in class_id_to_key])

In [13]:
len(class_id_to_key), len(key_to_classname)

(1000, 1861)

In [14]:
key_to_classname

{'?????????': 'dummy class for index 0',
 'n02119789': 'kit fox, Vulpes macrotis',
 'n02100735': 'English setter',
 'n02110185': 'Siberian husky',
 'n02096294': 'Australian terrier',
 'n02102040': 'English springer, English springer spaniel',
 'n02066245': 'grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus',
 'n02509815': 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens',
 'n02124075': 'Egyptian cat',
 'n02417914': 'ibex, Capra ibex',
 'n02123394': 'Persian cat',
 'n02125311': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor',
 'n02423022': 'gazelle',
 'n02346627': 'porcupine, hedgehog',
 'n02077923': 'sea lion',
 'n02110063': 'malamute, malemute, Alaskan malamute',
 'n02447366': 'badger',
 'n02109047': 'Great Dane',
 'n02089867': 'Walker hound, Walker foxhound',
 'n02102177': 'Welsh springer spaniel',
 'n02091134': 'whippet',
 'n02092002': 'Scottish deerhound, deerhound',
 'n02071294': 'killer whale, killer, or

In [15]:
class_id_to_key

array(['n01440764', 'n01443537', 'n01484850', 'n01491361', ..., 'n13052670', 'n13054560', 'n13133613', 'n15075141'],
      dtype='<U9')

In [16]:
classes = [key_to_classname[key] for key in class_id_to_key]

In [17]:
len(classes), classes[:5]

(1000,
 ['tench, Tinca tinca',
  'goldfish, Carassius auratus',
  'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
  'tiger shark, Galeocerdo cuvieri',
  'hammerhead, hammerhead shark'])